<a href="https://colab.research.google.com/github/Soorovickyy/facenet-pytorch/blob/main/fine_turning_no_cropped_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install facenet-pytorch

     |████████████████████████████████| 1.9 MB 4.9 MB/s 


In [2]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_dir = '/content/drive/MyDrive/face_recognition/data'
batch_size = 32

In [5]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

In [6]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True)

In [ ]:
for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')

del mtcnn

In [8]:
epochs = 8
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
)
optimizer = optim.Adam(resnet.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = MultiStepLR(optimizer, [3, 6])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.9 * len(img_inds))]
val_inds = img_inds[int(0.9 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

  0%|          | 0.00/107M [00:00<?, ?B/s]

In [9]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True,
        writer=writer
    )

writer.close()



Initial
----------
Valid |    10/10   | loss:    0.7169 | fps:    5.5735 | acc:    0.5096   

Epoch 1/8
----------
Train |    86/86   | loss:    0.4432 | fps:    3.2148 | acc:    0.8091   
Valid |    10/10   | loss:    0.3533 | fps:   13.8017 | acc:    0.8429   

Epoch 2/8
----------
Train |    86/86   | loss:    0.2841 | fps:    4.5471 | acc:    0.8854   
Valid |    10/10   | loss:    0.4962 | fps:   14.1110 | acc:    0.7667   

Epoch 3/8
----------
Train |    86/86   | loss:    0.2272 | fps:    4.5688 | acc:    0.9092   
Valid |    10/10   | loss:    0.2694 | fps:   14.1024 | acc:    0.9058   

Epoch 4/8
----------
Train |    86/86   | loss:    0.1309 | fps:    4.5759 | acc:    0.9509   
Valid |    10/10   | loss:    0.1913 | fps:   14.0724 | acc:    0.9156   

Epoch 5/8
----------
Train |    86/86   | loss:    0.1025 | fps:    4.5833 | acc:    0.9633   
Valid |    10/10   | loss:    0.1860 | fps:   14.1206 | acc:    0.9210   

Epoch 6/8
----------
Train |    86/86   | loss:    0.0